In [1]:
import os
import sys
from tqdm import tqdm_notebook
from datetime import datetime, timezone
import pytz
import pandas as pd
import numpy as np
from torchvision import models
import torch.nn as nn
import torch
import torch.optim as optim
## It's better to ensure the current path
os.chdir('../doc')
sys.path.append('../lib')

# Getting Started:
## A simple driving model training and evaluation pipeline using the Drive360 dataset and PyTorch.

## Loading data from Drive360 dataset.

The **dataset.py** file contains the 3 classes necessary for creating a Drive360Loader. Using the **config.json** file to specify the location of the csv and data directory, we can generate phase (train, validation, test) specific data loaders that can output samples from each set. Adjust the **dataset.py** to your preferred training framework.

In [2]:
import json
from dataset import Drive360Loader

# load the config.json file that specifies data 
# location parameters and other hyperparameters 
# required.
config = json.load(open('./config.json'))

normalize_targets = config['target']['normalize']
target_mean = config['target']['mean']
target_std = config['target']['std']

# create a train, validation and test data loader
train_loader = Drive360Loader(config, 'train')
validation_loader = Drive360Loader(config, 'validation')
test_loader = Drive360Loader(config, 'test')

# print the data (keys) available for use. See full 
# description of each data type in the documents.
print('Loaded train loader with the following data available as a dict.')
print(train_loader.drive360.dataframe.keys())

Phase: train # of data: 78292
Phase: validation # of data: 5151
Phase: test # of data: 14010
Loaded train loader with the following data available as a dict.
Index(['cameraRight', 'cameraFront', 'cameraRear', 'cameraLeft', 'canSteering',
       'canSpeed', 'chapter'],
      dtype='object')


In [ ]:
os.getcwd()

## Training a basic driving model

Create your driving model. This is specific to your learning framework. 

Below we give a very basic dummy model that uses the front facing camera and a resnet34 + LSTM architecture to predict canSteering and canSpeed. 

In [3]:
from Non_local_pytorch.lib.non_local_embedded_gaussian import NONLocalBlock2D
from model import FullDataModel

### Training and validation process

In [ ]:
def train_nn(train_loader, validation_loader, model, optimizer, 
             criterion, epochs=5, validation=True, load_path='', save_path='', print_freq = 200):
    '''Training the model
    Args:
        validation: boolean, whether process validation
        load_path: string, the model weights file to load
        save_path: string, the model weights file to save
    Returns:
    '''
    
    if torch.cuda.is_available():
        model = model.cuda()
        # model = nn.DataParallel(model)
        
    if os.path.exists(load_path):
        print('='*10 + 'loading weights from ' + load_path + '='*10)
        checkpoint = torch.load(load_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        print('='*10 + 'loading finished' + '='*10)
    else:
        print('load_path does not exist!')
        
    
    train_metrics = {
        'angle_loss': {},
        'speed_loss': {}
    }
    validation_metrics = {
        'angle_loss': {},
        'speed_loss': {}
    }
    for epoch in range(epochs):
        print('='*10 + 'start ' + str(epoch+1) + ' epoch' + '='*10)
        model.train()
        angle_loss = 0.0
        speed_loss = 0.0
        since = datetime.now()
        cnt = 0
        for batch_idx, (data, target) in enumerate(tqdm_notebook(train_loader)):
            optimizer.zero_grad()
            prediction = model(data)
            loss1 = criterion(prediction['canSteering'], target['canSteering'].cuda())
            loss2 = criterion(prediction['canSpeed'], target['canSpeed'].cuda())
            loss = loss1 + loss2
            loss.backward()
            optimizer.step()
            
            # print statistics
            angle_loss += loss1.item()
            speed_loss += loss2.item()
            cnt += 1
#             if batch_idx > 5:
#                 break
            if (batch_idx+1) % print_freq == 0:
                if normalize_targets:
                    angle_loss = (angle_loss * target_std['canSteering']**2) / cnt
                    speed_loss = (speed_loss * target_std['canSpeed']**2) / cnt
                else:
                    angle_loss /= cnt
                    speed_loss /= cnt
                train_metrics['angle_loss'].setdefault(str(epoch), []).append(angle_loss)
                train_metrics['speed_loss'].setdefault(str(epoch), []).append(speed_loss)
                print('[epoch: %d, batch: %5d] time: %.2f angle_loss: %.2f speed_loss: %.2f' %
                      (epoch + 1, batch_idx + 1, (datetime.now() - since).total_seconds(), angle_loss, speed_loss))
                angle_loss = 0.0
                speed_loss = 0.0
                since = datetime.now()
                cnt = 0
        print('='*10 + 'saving the model to' + save_path + '='*10)
        torch.save({
            "model_state_dict":model.state_dict(),
            "angle_loss": angle_loss,
            "speed_loss": speed_loss,
            "optimizer_state_dict":optimizer.state_dict(),
            "epoch":epoch
            }, save_path)
        print('saving success!')
        if validation:
            print('='*10 + 'starting validation' + '='*10)
            model.eval()
            with torch.no_grad():
                for batch_idx, (data, target) in enumerate(tqdm_notebook(validation_loader)):
                    if torch.cuda.is_available():
                        for w in ['canSteering', 'canSpeed']:
                            target[w].cuda()
                    prediction = model(data)
                    mse1 = (np.square(prediction['canSteering'].cpu() - target['canSteering'].cpu())).mean()
                    mse2 = (np.square(prediction['canSpeed'].cpu() - target['canSpeed'].cpu())).mean()
                    if normalize_targets:
                        mse1 = mse1 * target_std['canSteering'] ** 2
                        mse2 = mse2 * target_std['canSpeed'] ** 2
                    validation_metrics['angle_loss'].setdefault(str(epoch), []).append(mse1)
                    validation_metrics['speed_loss'].setdefault(str(epoch), []).append(mse2)
            print('angle_loss: %.2f speed_loss: %.2f' % (np.mean(validation_metrics['angle_loss'][str(epoch)]), 
                  np.mean(validation_metrics['speed_loss'][str(epoch)])))
            print('='*10 + 'validation finished' + '='*10)
    return train_metrics, validation_metrics

In [ ]:
NOW = datetime.now(pytz.timezone('US/eastern')).strftime("%m-%d-%H-%M")
MODEL_NAME = 'full_data'

if not os.path.isdir(os.path.join('../output', MODEL_NAME)):
    os.mkdir(os.path.join('../output', MODEL_NAME))

LOAD_PATH = os.path.join('../output', MODEL_NAME, '.pth')
SAVE_PATH = os.path.join('../output', MODEL_NAME, NOW + '_' + MODEL_NAME + '.pth')

model = FullDataModel()
criterion =nn.MSELoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001)

result = train_nn(train_loader, validation_loader, model, optimizer, criterion, epochs=10, 
                  validation=True, load_path=LOAD_PATH, save_path=SAVE_PATH, print_freq=100)


## Creating a submission file.

In [4]:
def add_results(results, output):
    steering = np.squeeze(output['canSteering'].cpu().data.numpy())
    speed = np.squeeze(output['canSpeed'].cpu().data.numpy())
    if normalize_targets:
        steering = (steering*target_std['canSteering'])+target_mean['canSteering']
        speed = (speed*target_std['canSpeed'])+target_mean['canSpeed']
    if np.isscalar(steering):
        steering = [steering]
    if np.isscalar(speed):
        speed = [speed]
    results['canSteering'].extend(steering)
    results['canSpeed'].extend(speed)

We use pandas to create a submission file which is simply a 2-column csv with a canSteering and canSpeed prediction for each row in the **drive360_test.csv** a total of 305437 rows/predictions not including the header. See the **sample_submission.csv** file as an example.

IMPORTANT: for the test phase indices will start 10s (100 samples) into each chapter this is to allow challenge participants to experiment with different temporal settings of data input. If challenge participants have a greater temporal length than 10s for each training sample, then they must write a custom function here. Please check out the **dataset.py** file for additional explanation.

In [5]:
results = {'canSteering': [],
          'canSpeed': []}

model = FullDataModel()
model.cuda()
MODEL_NAME = 'full_data'
LOAD_PATH = os.path.join('../output', MODEL_NAME, '10-17-17-15_full_data.pth')
checkpoint = torch.load(LOAD_PATH)
model.load_state_dict(checkpoint['model_state_dict'])

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tqdm_notebook(test_loader)):
        prediction = model(data)
        add_results(results, prediction)

df = pd.DataFrame.from_dict(results)

In [6]:
## linear interpolation
from scipy.interpolate import interp1d

file = os.path.join('../output', MODEL_NAME, 'submission.csv')
output = {
    'canSteering': [],
    'canSpeed': []
}
test_sample = pd.read_csv('../data/Sample2/test_sample2.csv')
curr_list = test_sample['chapter'].value_counts()-4
test_full = pd.read_csv('../data/test_full.csv')
target_list = test_full['chapter'].value_counts()-100
k = 0
for ch in test_sample['chapter'].unique():
    curr_num = curr_list[ch]
    target_num = target_list[ch]
    x = list(range(100, 100+20*curr_num, 20))
    # x.insert(0, 0)
    x.append(target_num+100)
    

    newx = list(range(101, target_num+101))
    y1, y2 = list(df.iloc[k:(k+curr_num),0]), list(df.iloc[k:(k+curr_num),1])
    #y1.insert(0, y1[0])
    #y1.insert(0, y1[0])
    y1.append(y1[-1])
    f1 = interp1d(x, y1, kind='linear')
    output['canSteering'].extend(f1(newx))
    
    #y2.insert(0, y2[0])
    #y2.insert(0, y2[0])
    y2.append(y2[-1])
    f2 = interp1d(x, y2, kind='linear')
    output['canSpeed'].extend(f2(newx))
    k += curr_num
    
output_df = pd.DataFrame(output)
print(len(output_df))

279863


In [7]:
output_df.to_csv(file, index=False)

In [8]:
output_df.head(50)

,canSteering,canSpeed
0,-7.357899,14.762022
1,-7.315518,14.745377
2,-7.273137,14.728731
3,-7.230756,14.712085
4,-7.188376,14.695439
5,-7.145995,14.678793
6,-7.103614,14.662147
7,-7.061233,14.645501
8,-7.018852,14.628855
9,-6.976471,14.612209


In [10]:
ori_submission = []
ori_submission.append(pd.read_csv('../output/full_data/model2_epoch5_1095.csv'))
ori_submission.append(pd.read_csv('../output/full_data/model2_epoch10_1028.csv'))
ori_submission.append(pd.read_csv('../output/full_data/model2_epoch15_1017.csv'))
ori_submission.append(pd.read_csv('../output/full_data/model1_epoch10_1114.csv'))
ori_submission.append(pd.read_csv('../output/full_data/model1_epoch15_1124.csv'))

submit = {
    'canSteering': [],
    'canSpeed': []
}

submit['canSteering'] = (ori_submission[0]['canSteering'] + 
                         ori_submission[1]['canSteering'] + 
                         ori_submission[2]['canSteering'] +
                         ori_submission[2]['canSteering'] +
                         ori_submission[2]['canSteering'])/len(ori_submission)
submit['canSpeed'] = (ori_submission[0]['canSpeed'] + 
                      ori_submission[1]['canSpeed'] + 
                      ori_submission[2]['canSpeed'] +
                      ori_submission[2]['canSpeed'] +
                      ori_submission[2]['canSpeed'])/len(ori_submission)
submit = pd.DataFrame(submit)
MODEL_NAME = 'full_data'
file = os.path.join('../output', MODEL_NAME, 'submission_combined.csv')
submit.to_csv(file, index=False)

In [11]:
submit.head(50)

,canSteering,canSpeed
0,-7.979080,13.645448
1,-7.925353,13.629842
2,-7.871625,13.614235
3,-7.817897,13.598629
4,-7.764170,13.583023
5,-7.710442,13.567417
6,-7.656714,13.551811
7,-7.602987,13.536205
8,-7.549259,13.520599
9,-7.495531,13.504993


In [ ]:
!pip3 freeze > requirements.txt